# Intro 

In this notebook, we will learn how to use hail functions to annotate the Expression Modifier Score (EMS) for a given list of variant-gene pairs (vgs), 

assuming that readers do not have a cloud computing environment set up.

As a first step, let's download the EMS file in tsv format to local (see the [finucane lab page](http://finucanelab.org/data) for instructions).

Once we have the tsv (block-zipped) in local, we can read that in pandas:
(Note: Here we are directly reading the `tsv.gz` in the cloud using hail and hadoop, but this can be done simpler by `df = pd.read_csv(your_local_data_dir)`), as shown in the comment below.

In [1]:
#import other useful tools
import time as tm
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

tissue_name = "Whole_Blood"

###
#import and initialize hail 
import hail as hl
import hail.expr.aggregators as agg
hl.init(default_reference='GRCh38')
#and read the file with hail hadoop
fn = "gs://expression-modifier-score/public/tsv/ems_top_{0}.tsv.bgz".format(tissue_name)
with hl.hadoop_open(fn, 'r') as f:
    df = pd.read_csv(f, sep="\t")
###

#In local, this ### can be replaced with the following one line:
# df = pd.read_csv(your_local_file_path, sep="\t")

df.head()

Running on Apache Spark version 2.4.5
SparkUI available at http://qbwcluster-mpl-m.c.encode-uk-biobank.internal:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.26-2dcc3d963867
LOGGING: writing to /home/hail/hail-20200914-2207-0.2.26-2dcc3d963867.log


,v,g,rf_score_raw,ems,ems_normalized
0,chr10_100006504_T_C_b38,ENSG00000107554.16,0.98046,0.009245,761.83
1,chr10_100006605_T_C_b38,ENSG00000107554.16,0.87271,0.009629,793.46
2,chr10_100006744_C_G_b38,ENSG00000107554.16,0.94024,0.001634,134.62
3,chr10_100009635_T_G_b38,ENSG00000107554.16,0.91919,0.001803,148.62
4,chr10_100167711_G_A_b38,ENSG00000107566.13,0.92195,0.002227,183.52


Now that we have the EMS table as a pandas dataframe, filtering, aggregation and other major analyses are straightforward:

## Filtering
Example of filtering to variant-genes with `ems_normalized`>500:

In [2]:
dfhigh = df[df.ems_normalized>500]
dfhigh.head()

,v,g,rf_score_raw,ems,ems_normalized
0,chr10_100006504_T_C_b38,ENSG00000107554.16,0.98046,0.009245,761.83
1,chr10_100006605_T_C_b38,ENSG00000107554.16,0.87271,0.009629,793.46
9,chr10_100168530_TGA_T_b38,ENSG00000213341.10,0.87273,0.009629,793.46
11,chr10_100184325_G_C_b38,ENSG00000107566.13,0.89993,0.008127,669.68
12,chr10_100184325_G_C_b38,ENSG00000236308.1,0.89710,0.008082,665.98


## Aggregation
One might be interested in only the variants. We can aggregate the table to get the maximum EMS for a variant:

In [3]:
dfv = df.groupby("v").ems.agg(["max"])
dfv.head()

,max
v,
chr10_100006504_T_C_b38,0.009245
chr10_100006605_T_C_b38,0.009629
chr10_100006744_C_G_b38,0.001634
chr10_100009635_T_G_b38,0.001803
chr10_100167711_G_A_b38,0.002227


## Annotating EMS for a set of vgs
Here we will show an example of annotating EMS for a set of variant and genes of interest.

Let's assume we have a tsv file with 10000 rows of variant and genes:
(We will make that by a random sampling of the `df`)

In [4]:
vgs = df.sample(n=10000, random_state=1).reset_index()[["v","g"]]
vgs.head()

,v,g
0,chr21_41362487_T_G_b38,ENSG00000183486.12
1,chr17_73233212_C_T_b38,ENSG00000141219.15
2,chr2_74120467_G_C_b38,ENSG00000217702.2
3,chr22_20015967_C_CAG_b38,ENSG00000183597.15
4,chr17_78157921_T_A_b38,ENSG00000108639.7


We can annotate the EMS by join:

In [5]:
#index the both df with v and g
df.index = df.v + "_" + df.g
vgs.index = vgs.v + "_" + vgs.g
#join
vgs = vgs.join(df["ems"], how="left")
vgs.head()

,v,g,ems
chr21_41362487_T_G_b38_ENSG00000183486.12,chr21_41362487_T_G_b38,ENSG00000183486.12,0.001962
chr17_73233212_C_T_b38_ENSG00000141219.15,chr17_73233212_C_T_b38,ENSG00000141219.15,0.012074
chr2_74120467_G_C_b38_ENSG00000217702.2,chr2_74120467_G_C_b38,ENSG00000217702.2,0.001641
chr22_20015967_C_CAG_b38_ENSG00000183597.15,chr22_20015967_C_CAG_b38,ENSG00000183597.15,0.004186
chr17_78157921_T_A_b38_ENSG00000108639.7,chr17_78157921_T_A_b38,ENSG00000108639.7,0.004465


# End notes

This tutorial explained basic properties of EMS and how to annotate EMS to a set of variant-gene pairs in local python environment. Such approach would be simple and fast when the size of the variant-gene pairs are small and/or people are interested in a small subset of variant-genes with high EMS.


For users who are familiar with cloud computing using hail and seeking of large scale computation using EMS, we recommend using the cloud; the corresponding tutorial is available at `tutorial.ipynb`.